# Bayesian Temporal Matrix Factorization

**Published**: October 8, 2019

**Revised**: October 8, 2020

**Author**: Xinyu Chen [[**GitHub homepage**](https://github.com/xinychen)]

**Download**: This Jupyter notebook is at our GitHub repository. If you want to evaluate the code, please download the notebook from the [**transdim**](https://github.com/xinychen/transdim/blob/master/predictor/BTMF.ipynb) repository.

This notebook shows how to implement the Bayesian Temporal Matrix Factorization (BTMF), a fully Bayesian matrix factorization model, on some real-world data sets. To overcome the missing data problem in multivariate time series, BTMF takes into account both low-rank matrix structure and time series autoregression. For an in-depth discussion of BTMF, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Xinyu Chen, Lijun Sun (2019). <b>Bayesian temporal factorization for multidimensional time series prediction</b>. arXiv:1910.06366. <a href="https://arxiv.org/pdf/1910.06366.pdf" title="PDF"><b>[PDF]</b></a> 
</font>
</div>

## Problem Description

We assume a spatiotemporal setting for multidimensional time series data throughout this work. In general, modern spatiotemporal data sets collected from sensor networks can be organized as matrix time series. For example, we can denote by matrix $Y\in\mathbb{R}^{N\times T}$ a multivariate time series collected from $N$ locations/sensors on $T$ time points, with each row $$\boldsymbol{y}_{i}=\left(y_{i,1},y_{i,2},...,y_{i,t-1},y_{i,t},y_{i,t+1},...,y_{i,T}\right)$$
corresponding to the time series collected at location $i$.

As mentioned, making accurate predictions on incomplete time series is very challenging, while missing data problem is almost inevitable in real-world applications. Figure 1 illustrates the prediction problem for incomplete time series data. Here we use $(i,t)\in\Omega$ to index the observed entries in matrix $Y$.

<img src="../images/graphical_matrix_time_series.png" alt="drawing" width="500"/>

> **Figure 1**: Illustration of multivariate time series and the prediction problem in the presence of missing values (green: observed data; white: missing data; red: prediction).


In [1]:
import numpy as np
from numpy.linalg import inv as inv
from numpy.random import normal as normrnd
from numpy.random import multivariate_normal as mvnrnd
from scipy.linalg import khatri_rao as kr_prod
from scipy.stats import wishart
from scipy.stats import invwishart
from numpy.linalg import solve as solve
from numpy.linalg import cholesky as cholesky_lower
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

In [3]:
def sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau, beta0 = 1, vargin = 0):
    """Sampling N-by-R factor matrix W and its hyperparameters (mu_w, Lambda_w)."""
    
    dim1, rank = W.shape
    W_bar = np.mean(W, axis = 0)
    temp = dim1 / (dim1 + beta0)
    var_W_hyper = inv(np.eye(rank) + cov_mat(W, W_bar) + temp * beta0 * np.outer(W_bar, W_bar))
    var_Lambda_hyper = wishart.rvs(df = dim1 + rank, scale = var_W_hyper)
    var_mu_hyper = mvnrnd_pre(temp * W_bar, (dim1 + beta0) * var_Lambda_hyper)
    
    if dim1 * rank ** 2 > 1e+8:
        vargin = 1
    
    if vargin == 0:
        var1 = X.T
        var2 = kr_prod(var1, var1)
        var3 = (var2 @ tau_ind.T).reshape([rank, rank, dim1]) + var_Lambda_hyper[:, :, None]
        var4 = var1 @ tau_sparse_mat.T + (var_Lambda_hyper @ var_mu_hyper)[:, None]
        for i in range(dim1):
            W[i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
    elif vargin == 1:
        for i in range(dim1):
            pos0 = np.where(sparse_mat[i, :] != 0)
            Xt = X[pos0[0], :]
            var_mu = tau[i] * Xt.T @ sparse_mat[i, pos0[0]] + var_Lambda_hyper @ var_mu_hyper
            var_Lambda = tau[i] * Xt.T @ Xt + var_Lambda_hyper
            W[i, :] = mvnrnd_pre(solve(var_Lambda, var_mu), var_Lambda)
    
    return W

In [4]:
def mnrnd(M, U, V):
    """
    Generate matrix normal distributed random matrix.
    M is a m-by-n matrix, U is a m-by-m matrix, and V is a n-by-n matrix.
    """
    dim1, dim2 = M.shape
    X0 = np.random.randn(dim1, dim2)
    P = cholesky_lower(U)
    Q = cholesky_lower(V)
    
    return M + P @ X0 @ Q.T

def sample_var_coefficient(X, time_lags):
    dim, rank = X.shape
    d = time_lags.shape[0]
    tmax = np.max(time_lags)
    
    Z_mat = X[tmax : dim, :]
    Q_mat = np.zeros((dim - tmax, rank * d))
    for k in range(d):
        Q_mat[:, k * rank : (k + 1) * rank] = X[tmax - time_lags[k] : dim - time_lags[k], :]
    var_Psi0 = np.eye(rank * d) + Q_mat.T @ Q_mat
    var_Psi = inv(var_Psi0)
    var_M = var_Psi @ Q_mat.T @ Z_mat
    var_S = np.eye(rank) + Z_mat.T @ Z_mat - var_M.T @ var_Psi0 @ var_M
    Sigma = invwishart.rvs(df = rank + dim - tmax, scale = var_S)
    
    return mnrnd(var_M, var_Psi, Sigma), Sigma

In [5]:
def sample_factor_x(tau_sparse_mat, tau_ind, time_lags, W, X, A, Lambda_x):
    """Sampling T-by-R factor matrix X."""
    
    dim2, rank = X.shape
    tmax = np.max(time_lags)
    tmin = np.min(time_lags)
    d = time_lags.shape[0]
    A0 = np.dstack([A] * d)
    for k in range(d):
        A0[k * rank : (k + 1) * rank, :, k] = 0
    mat0 = Lambda_x @ A.T
    mat1 = np.einsum('kij, jt -> kit', A.reshape([d, rank, rank]), Lambda_x)
    mat2 = np.einsum('kit, kjt -> ij', mat1, A.reshape([d, rank, rank]))
    
    var1 = W.T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ tau_ind).reshape([rank, rank, dim2]) + Lambda_x[:, :, None]
    var4 = var1 @ tau_sparse_mat
    for t in range(dim2):
        Mt = np.zeros((rank, rank))
        Nt = np.zeros(rank)
        Qt = mat0 @ X[t - time_lags, :].reshape(rank * d)
        index = list(range(0, d))
        if t >= dim2 - tmax and t < dim2 - tmin:
            index = list(np.where(t + time_lags < dim2))[0]
        elif t < tmax:
            Qt = np.zeros(rank)
            index = list(np.where(t + time_lags >= tmax))[0]
        if t < dim2 - tmin:
            Mt = mat2.copy()
            temp = np.zeros((rank * d, len(index)))
            n = 0
            for k in index:
                temp[:, n] = X[t + time_lags[k] - time_lags, :].reshape(rank * d)
                n += 1
            temp0 = X[t + time_lags[index], :].T - np.einsum('ijk, ik -> jk', A0[:, :, index], temp)
            Nt = np.einsum('kij, jk -> i', mat1[index, :, :], temp0)
        
        var3[:, :, t] = var3[:, :, t] + Mt
        if t < tmax:
            var3[:, :, t] = var3[:, :, t] - Lambda_x + np.eye(rank)
        X[t, :] = mvnrnd_pre(solve(var3[:, :, t], var4[:, t] + Nt + Qt), var3[:, :, t])

    return X

In [6]:
def sample_precision_tau(sparse_mat, mat_hat, ind):
    var_alpha = 1e-6 + 0.5 * np.sum(ind, axis = 1)
    var_beta = 1e-6 + 0.5 * np.sum(((sparse_mat - mat_hat) ** 2) * ind, axis = 1)
    return np.random.gamma(var_alpha, 1 / var_beta)

In [7]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [8]:
def ar4cast(A, X, Sigma, time_lags, multi_step):
    dim, rank = X.shape
    d = time_lags.shape[0]
    X_new = np.append(X, np.zeros((multi_step, rank)), axis = 0)
    for t in range(multi_step):
        var = A.T @ X_new[dim + t - time_lags, :].reshape(rank * d)
        X_new[dim + t, :] = mvnrnd(var, Sigma)
    return X_new

#### BTMF Implementation



In [9]:
def BTMF(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter, multi_step = 1):
    """Bayesian Temporal Matrix Factorization, BTMF."""
    
    dim1, dim2 = sparse_mat.shape
    d = time_lags.shape[0]
    W = init["W"]
    X = init["X"]
    if np.isnan(sparse_mat).any() == False:
        ind = sparse_mat != 0
        pos_obs = np.where(ind)
        pos_test = np.where((dense_mat != 0) & (sparse_mat == 0))
    elif np.isnan(sparse_mat).any() == True:
        pos_test = np.where((dense_mat != 0) & (np.isnan(sparse_mat)))
        ind = ~np.isnan(sparse_mat)
        pos_obs = np.where(ind)
        sparse_mat[np.isnan(sparse_mat)] = 0
    dense_test = dense_mat[pos_test]
    del dense_mat
    tau = np.ones(dim1)
    W_plus = np.zeros((dim1, rank, gibbs_iter))
    A_plus = np.zeros((rank * d, rank, gibbs_iter))
    tau_plus = np.zeros((dim1, gibbs_iter))
    Sigma_plus = np.zeros((rank, rank, gibbs_iter))
    temp_hat = np.zeros(len(pos_test[0]))
    show_iter = 500
    mat_hat_plus = np.zeros((dim1, dim2))
    X_plus = np.zeros((dim2 + multi_step, rank, gibbs_iter))
    mat_new_plus = np.zeros((dim1, multi_step))
    for it in range(burn_iter + gibbs_iter):
        tau_ind = tau[:, None] * ind
        tau_sparse_mat = tau[:, None] * sparse_mat
        W = sample_factor_w(tau_sparse_mat, tau_ind, W, X, tau, beta0 = 1, vargin = 0)
        A, Sigma = sample_var_coefficient(X, time_lags)
        X = sample_factor_x(tau_sparse_mat, tau_ind, time_lags, W, X, A, inv(Sigma))
        mat_hat = W @ X.T
        tau = sample_precision_tau(sparse_mat, mat_hat, ind)
        temp_hat += mat_hat[pos_test]
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            temp_hat = np.zeros(len(pos_test[0]))
            print()
        if it + 1 > burn_iter:
            W_plus[:, :, it - burn_iter] = W
            A_plus[:, :, it - burn_iter] = A
            Sigma_plus[:, :, it - burn_iter] = Sigma
            tau_plus[:, it - burn_iter] = tau
            mat_hat_plus += mat_hat
            X0 = ar4cast(A, X, Sigma, time_lags, multi_step)
            X_plus[:, :, it - burn_iter] = X0
            mat_new_plus += W @ X0[dim2 : dim2 + multi_step, :].T
    mat_hat = mat_hat_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_test, mat_hat[:, : dim2][pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_test, mat_hat[:, : dim2][pos_test])))
    print()
    mat_hat = np.append(mat_hat, mat_new_plus / gibbs_iter, axis = 1)
    mat_hat[mat_hat < 0] = 0
    
    return mat_hat, W_plus, X_plus, A_plus, Sigma_plus, tau_plus

In [10]:
def sample_factor_x_partial(tau_sparse_mat, tau_ind, time_lags, W, X, A, Lambda_x, back_step):
    """Sampling T-by-R factor matrix X."""
    
    dim2, rank = X.shape
    tmax = np.max(time_lags)
    tmin = np.min(time_lags)
    d = time_lags.shape[0]
    A0 = np.dstack([A] * d)
    for k in range(d):
        A0[k * rank : (k + 1) * rank, :, k] = 0
    mat0 = Lambda_x @ A.T
    mat1 = np.einsum('kij, jt -> kit', A.reshape([d, rank, rank]), Lambda_x)
    mat2 = np.einsum('kit, kjt -> ij', mat1, A.reshape([d, rank, rank]))
    
    var1 = W.T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ tau_ind[:, - back_step :]).reshape([rank, rank, back_step]) + Lambda_x[:, :, None]
    var4 = var1 @ tau_sparse_mat[:, - back_step :]
    for t in range(dim2 - back_step, dim2):
        Mt = np.zeros((rank, rank))
        Nt = np.zeros(rank)
        Qt = mat0 @ X[t - time_lags, :].reshape(rank * d)
        index = list(range(0, d))
        if t >= dim2 - tmax and t < dim2 - tmin:
            index = list(np.where(t + time_lags < dim2))[0]
        if t < dim2 - tmin:
            Mt = mat2.copy()
            temp = np.zeros((rank * d, len(index)))
            n = 0
            for k in index:
                temp[:, n] = X[t + time_lags[k] - time_lags, :].reshape(rank * d)
                n += 1
            temp0 = X[t + time_lags[index], :].T - np.einsum('ijk, ik -> jk', A0[:, :, index], temp)
            Nt = np.einsum('kij, jk -> i', mat1[index, :, :], temp0)
        var3[:, :, t + back_step - dim2] = var3[:, :, t + back_step - dim2] + Mt
        X[t, :] = mvnrnd_pre(solve(var3[:, :, t + back_step - dim2], 
                                   var4[:, t + back_step - dim2] + Nt + Qt), var3[:, :, t + back_step - dim2])
    return X

In [11]:
def BTMF_partial(dense_mat, sparse_mat, init, rank, time_lags, burn_iter, gibbs_iter, multi_step = 1):
    """Bayesian Temporal Matrix Factorization, BTMF."""
    
    dim1, dim2 = sparse_mat.shape
    W_plus = init["W_plus"]
    X_plus = init["X_plus"]
    A_plus = init["A_plus"]
    Sigma_plus = init["Sigma_plus"]
    tau_plus = init["tau_plus"]
    if np.isnan(sparse_mat).any() == False:
        ind = sparse_mat != 0
        pos_obs = np.where(ind)
    elif np.isnan(sparse_mat).any() == True:
        ind = ~np.isnan(sparse_mat)
        pos_obs = np.where(ind)
        sparse_mat[np.isnan(sparse_mat)] = 0
    X_new_plus = np.zeros((dim2 + multi_step, rank, gibbs_iter))
    mat_new_plus = np.zeros((dim1, multi_step))
    back_step = 10 * multi_step
    for it in range(gibbs_iter):
        tau_ind = tau_plus[:, it][:, None] * ind
        tau_sparse_mat = tau_plus[:, it][:, None] * sparse_mat
        X = sample_factor_x_partial(tau_sparse_mat, tau_ind, time_lags, W_plus[:, :, it], 
                                    X_plus[:, :, it], A_plus[:, :, it], inv(Sigma_plus[:, :, it]), back_step)
        X0 = ar4cast(A_plus[:, :, it], X, Sigma_plus[:, :, it], time_lags, multi_step)
        X_new_plus[:, :, it] = X0
        mat_new_plus += W_plus[:, :, it] @ X0[- multi_step :, :].T
    mat_hat = mat_new_plus / gibbs_iter
    mat_hat[mat_hat < 0] = 0
    
    return mat_hat, W_plus, X_new_plus, A_plus, Sigma_plus, tau_plus

In [12]:
from ipywidgets import IntProgress
from IPython.display import display

def BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter):
    dim1, T = dense_mat
    start_time = T - pred_step
    max_count = int(np.ceil(pred_step / multi_step))
    mat_hat = np.zeros((dim1, max_count * multi_step))
    f = IntProgress(min = 0, max = max_count) # instantiate the bar
    display(f) # display the bar
    for t in range(max_count):
        if t == 0:
            init = {"W": 0.1 * np.random.randn(dim1, rank), "X": 0.1 * np.random.randn(start_time, rank)}
            mat, W, X_new, A, Sigma, tau = BTMF(dense_mat[:, 0 : start_time], 
                sparse_mat[:, 0 : start_time], init, rank, time_lags, burn_iter, gibbs_iter, multi_step)
        else:
            init = {"W_plus": W, "X_plus": X_new, "A_plus": A, "Sigma_plus": Sigma, "tau_plus": tau}
            mat, W, X_new, A, Sigma, tau = BTMF_partial(dense_mat[:, 0 : start_time + t * multi_step], 
                sparse_mat[:, 0 : start_time + t * multi_step], init, rank, time_lags, 
                                                        burn_iter, gibbs_iter, multi_step)
        mat_hat[:, t * multi_step : (t + 1) * multi_step] = mat[:, - multi_step :]
        f.value = t
    small_dense_mat = dense_mat[:, start_time : T]
    pos = np.where(small_dense_mat != 0)
    print('Prediction MAPE: {:.6}'.format(compute_mape(small_dense_mat[pos], mat_hat[pos])))
    print('Prediction RMSE: {:.6}'.format(compute_rmse(small_dense_mat[pos], mat_hat[pos])))
    print()
    return mat_hat

## Evaluation on Guangzhou Speed Data



**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Test on original data


In [13]:
import scipy.io
import warnings
warnings.simplefilter('ignore')

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
sparse_mat = dense_mat.copy()

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 144
- Time lags: {1, 2, 144, 144 + 1, 144 + 2, 7 * 144, 7 * 144 + 1, 7 * 144 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [14]:
import time
rank = 10
pred_step = 7 * 144
time_lags = np.array([1, 2, 3, 144, 145, 146, 7 * 144, 7 * 144 + 1, 7 * 144 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=504)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.110104
Prediction RMSE: 4.44345

Running time: 6904 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=252)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.112229
Prediction RMSE: 4.5363

Running time: 4315 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.123838
Prediction RMSE: 4.99738

Running time: 1950 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=21)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.122124
Prediction RMSE: 4.92829

Running time: 1919 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=19)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.125164
Prediction RMSE: 5.00299

Running time: 1897 seconds



**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Non-random missing (NM)
- 40% missing rate


In [ ]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_matrix.mat')['random_matrix']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 144
- Time lags: {1, 2, 144, 144 + 1, 144 + 2, 7 * 144, 7 * 144 + 1, 7 * 144 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [ ]:
import time
rank = 10
pred_step = 7 * 144
time_lags = np.array([1, 2, 3, 144, 145, 146, 7 * 144, 7 * 144 + 1, 7 * 144 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Random missing (RM)
- 40% missing rate


In [17]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 144
- Time lags: {1, 2, 144, 144 + 1, 144 + 2, 7 * 144, 7 * 144 + 1, 7 * 144 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [18]:
import time
rank = 10
pred_step = 7 * 144
time_lags = np.array([1, 2, 3, 144, 145, 146, 7 * 144, 7 * 144 + 1, 7 * 144 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=504)

Iter: 500
MAPE: 0.0993829
RMSE: 4.18949

Iter: 1000
MAPE: 0.0989114
RMSE: 4.1819

Imputation MAPE: 0.0988468
Imputation RMSE: 4.17973

Prediction MAPE: 0.112213
Prediction RMSE: 4.51016

Running time: 3492 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=252)

Iter: 500
MAPE: 0.0997469
RMSE: 4.19924

Iter: 1000
MAPE: 0.0990423
RMSE: 4.18674

Imputation MAPE: 0.0989876
Imputation RMSE: 4.18463

Prediction MAPE: 0.115235
Prediction RMSE: 4.63405

Running time: 2571 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.0993313
RMSE: 4.18762

Iter: 1000
MAPE: 0.0989223
RMSE: 4.18231

Imputation MAPE: 0.0988696
Imputation RMSE: 4.18026

Prediction MAPE: 0.125404
Prediction RMSE: 5.04455

Running time: 1836 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=21)

Iter: 500
MAPE: 0.0994085
RMSE: 4.18929

Iter: 1000
MAPE: 0.0989061
RMSE: 4.18137

Imputation MAPE: 0.0988842
Imputation RMSE: 4.18015

Prediction MAPE: 0.126043
Prediction RMSE: 5.07421

Running time: 1805 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=19)

Iter: 500
MAPE: 0.0992844
RMSE: 4.18683

Iter: 1000
MAPE: 0.0988439
RMSE: 4.17924

Imputation MAPE: 0.0987823
Imputation RMSE: 4.17773

Prediction MAPE: 0.128917
Prediction RMSE: 5.12116

Running time: 1802 seconds



**Scenario setting**:

- Tensor size: $214\times 61\times 144$ (road segment, day, time of day)
- Random missing (RM)
- 60% missing rate


In [19]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Guangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.6

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 144
- Time lags: {1, 2, 144, 144 + 1, 144 + 2, 7 * 144, 7 * 144 + 1, 7 * 144 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [20]:
import time
rank = 10
pred_step = 7 * 144
time_lags = np.array([1, 2, 3, 144, 145, 146, 7 * 144, 7 * 144 + 1, 7 * 144 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=504)

Iter: 500
MAPE: 0.100833
RMSE: 4.22916

Iter: 1000
MAPE: 0.10005
RMSE: 4.2153

Imputation MAPE: 0.100027
Imputation RMSE: 4.21426

Prediction MAPE: 0.114409
Prediction RMSE: 4.58439

Running time: 3368 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=252)

Iter: 500
MAPE: 0.101056
RMSE: 4.23486

Iter: 1000
MAPE: 0.10016
RMSE: 4.21878

Imputation MAPE: 0.100078
Imputation RMSE: 4.21675

Prediction MAPE: 0.11659
Prediction RMSE: 4.65573

Running time: 2536 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.101084
RMSE: 4.23696

Iter: 1000
MAPE: 0.100128
RMSE: 4.21826

Imputation MAPE: 0.100101
Imputation RMSE: 4.21737

Prediction MAPE: 0.127483
Prediction RMSE: 5.06699

Running time: 1834 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=21)

Iter: 500
MAPE: 0.101008
RMSE: 4.23385

Iter: 1000
MAPE: 0.100163
RMSE: 4.21887

Imputation MAPE: 0.100063
Imputation RMSE: 4.21606

Prediction MAPE: 0.12774
Prediction RMSE: 5.12949

Running time: 1836 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=19)

Iter: 500
MAPE: 0.101061
RMSE: 4.23526

Iter: 1000
MAPE: 0.100138
RMSE: 4.21821

Imputation MAPE: 0.100067
Imputation RMSE: 4.21589

Prediction MAPE: 0.128683
Prediction RMSE: 5.11977

Running time: 1820 seconds



## Evaluation on Hangzhou Flow Data



**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Test on original data


In [21]:
import scipy.io
import warnings
warnings.simplefilter('ignore')

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
sparse_mat = dense_mat.copy()

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 108
- Time lags: {1, 2, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [22]:
import time
rank = 10
pred_step = 7 * 108
time_lags = np.array([1, 2, 3, 108, 109, 110, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=378)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.25531
Prediction RMSE: 43.1972

Running time: 625 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=189)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.285376
Prediction RMSE: 43.393

Running time: 583 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.344838
Prediction RMSE: 51.3397

Running time: 771 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=16)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.321935
Prediction RMSE: 48.6428

Running time: 646 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=14)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.30817
Prediction RMSE: 47.9637

Running time: 783 seconds



**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Non-random missing (NM)
- 40% missing rate


In [23]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
random_matrix = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_matrix.mat')['random_matrix']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(tensor.shape)
for i1 in range(tensor.shape[0]):
    for i2 in range(tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
binary_mat = binary_tensor.reshape([binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]])
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 108
- Time lags: {1, 2, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [24]:
import time
rank = 10
pred_step = 7 * 108
time_lags = np.array([1, 2, 3, 108, 109, 110, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=378)

Iter: 500
MAPE: 0.711485
RMSE: 180.212

Iter: 1000
MAPE: 0.808173
RMSE: 239.82

Imputation MAPE: 0.718951
Imputation RMSE: 213.655

Prediction MAPE: 0.281785
Prediction RMSE: 50.1093

Running time: 709 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=189)

Iter: 500
MAPE: 0.275665
RMSE: 48.4204

Iter: 1000
MAPE: 0.241641
RMSE: 44.8259

Imputation MAPE: 0.242374
Imputation RMSE: 44.9511

Prediction MAPE: 0.269033
Prediction RMSE: 48.3854

Running time: 581 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.244632
RMSE: 46.0329

Iter: 1000
MAPE: 0.240758
RMSE: 44.9312

Imputation MAPE: 0.240775
Imputation RMSE: 44.9519

Prediction MAPE: 0.367984
Prediction RMSE: 55.3295

Running time: 541 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=16)

Iter: 500
MAPE: 0.24346
RMSE: 44.8582

Iter: 1000
MAPE: 0.241779
RMSE: 44.8746

Imputation MAPE: 0.24415
Imputation RMSE: 44.3115

Prediction MAPE: 0.384108
Prediction RMSE: 55.9597

Running time: 542 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=14)

Iter: 500
MAPE: 0.240889
RMSE: 44.6134

Iter: 1000
MAPE: 0.24082
RMSE: 44.9075

Imputation MAPE: 0.243892
Imputation RMSE: 44.916

Prediction MAPE: 0.322542
Prediction RMSE: 51.1474

Running time: 536 seconds



**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Random missing (RM)
- 40% missing rate


In [25]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.4

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 108
- Time lags: {1, 2, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [26]:
import time
rank = 10
pred_step = 7 * 108
time_lags = np.array([1, 2, 3, 108, 109, 110, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=378)

Iter: 500
MAPE: 0.226961
RMSE: 43.2707

Iter: 1000
MAPE: 0.229531
RMSE: 42.9285

Imputation MAPE: 0.230566
Imputation RMSE: 42.9642

Prediction MAPE: 0.261847
Prediction RMSE: 42.361

Running time: 665 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=189)

Iter: 500
MAPE: 0.229738
RMSE: 43.6264

Iter: 1000
MAPE: 0.231535
RMSE: 42.9413

Imputation MAPE: 0.233564
Imputation RMSE: 42.9716

Prediction MAPE: 0.277802
Prediction RMSE: 43.5395

Running time: 598 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.227151
RMSE: 43.4306

Iter: 1000
MAPE: 0.231705
RMSE: 42.9291

Imputation MAPE: 0.232605
Imputation RMSE: 42.8047

Prediction MAPE: 0.399002
Prediction RMSE: 56.3718

Running time: 545 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=16)

Iter: 500
MAPE: 0.226823
RMSE: 43.3191

Iter: 1000
MAPE: 0.231322
RMSE: 42.9051

Imputation MAPE: 0.233432
Imputation RMSE: 42.9269

Prediction MAPE: 0.349713
Prediction RMSE: 53.8556

Running time: 548 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=14)

Iter: 500
MAPE: 0.227789
RMSE: 43.5207

Iter: 1000
MAPE: 0.23074
RMSE: 42.9006

Imputation MAPE: 0.232072
Imputation RMSE: 42.8578

Prediction MAPE: 0.489229
Prediction RMSE: 66.3897

Running time: 542 seconds



**Scenario setting**:

- Tensor size: $80\times 25\times 108$ (metro station, day, time of day)
- Random missing (RM)
- 60% missing rate


In [27]:
import scipy.io

tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/tensor.mat')['tensor']
random_tensor = scipy.io.loadmat('../datasets/Hangzhou-data-set/random_tensor.mat')['random_tensor']
dense_mat = tensor.reshape([tensor.shape[0], tensor.shape[1] * tensor.shape[2]])
missing_rate = 0.6

## Random missing (RM)
binary_mat = (np.round(random_tensor + 0.5 - missing_rate)
              .reshape([random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]]))
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 108
- Time lags: {1, 2, 108, 108 + 1, 108 + 2, 7 * 108, 7 * 108 + 1, 7 * 108 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [28]:
import time
rank = 10
pred_step = 7 * 108
time_lags = np.array([1, 2, 3, 108, 109, 110, 7 * 108, 7 * 108 + 1, 7 * 108 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=378)

Iter: 500
MAPE: 0.245447
RMSE: 46.2954

Iter: 1000
MAPE: 0.251494
RMSE: 45.6868

Imputation MAPE: 0.253889
Imputation RMSE: 45.6802

Prediction MAPE: 0.270133
Prediction RMSE: 44.1042

Running time: 670 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=189)

Iter: 500
MAPE: 0.247711
RMSE: 46.4123

Iter: 1000
MAPE: 0.249532
RMSE: 45.6559

Imputation MAPE: 0.247246
Imputation RMSE: 44.8514

Prediction MAPE: 0.268741
Prediction RMSE: 46.2648

Running time: 596 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.248345
RMSE: 46.5185

Iter: 1000
MAPE: 0.245469
RMSE: 44.8312

Imputation MAPE: 0.250308
Imputation RMSE: 44.8471

Prediction MAPE: 0.360375
Prediction RMSE: 51.8993

Running time: 541 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=16)

Iter: 500
MAPE: 0.241355
RMSE: 46.0682

Iter: 1000
MAPE: 0.245772
RMSE: 44.8256

Imputation MAPE: 0.246172
Imputation RMSE: 44.8468

Prediction MAPE: 0.315255
Prediction RMSE: 51.169

Running time: 549 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=14)

Iter: 500
MAPE: 0.252981
RMSE: 46.5808

Iter: 1000
MAPE: 0.250108
RMSE: 45.7151

Imputation MAPE: 0.24693
Imputation RMSE: 44.6762

Prediction MAPE: 0.352228
Prediction RMSE: 52.561

Running time: 549 seconds



## Evaluation on Seattle Speed Data



**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Test on original data


In [29]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
dense_mat = dense_mat.values
sparse_mat = dense_mat.copy()

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 288
- Time lags: {1, 2, 288, 288 + 1, 288 + 2, 7 * 288, 7 * 288 + 1, 7 * 288 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [30]:
import time
rank = 10
pred_step = 7 * 288
time_lags = np.array([1, 2, 3, 288, 289, 290, 7 * 288, 7 * 288 + 1, 7 * 288 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=1008)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.103915
Prediction RMSE: 5.63268

Running time: 4625 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=504)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.108246
Prediction RMSE: 5.8263

Running time: 3034 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.144885
Prediction RMSE: 7.69699

Running time: 1876 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=42)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.143099
Prediction RMSE: 7.54653

Running time: 1830 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=38)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.1424
Prediction RMSE: 7.6484

Running time: 1917 seconds



**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Non-random missing (NM)
- 40% missing rate


In [31]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
NM_mat = pd.read_csv('../datasets/Seattle-data-set/NM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
NM_mat = NM_mat.values
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros((dense_mat.shape[0], 28, 288))
for i1 in range(binary_tensor.shape[0]):
    for i2 in range(binary_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(NM_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_tensor.reshape([dense_mat.shape[0], dense_mat.shape[1]]))

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 288
- Time lags: {1, 2, 288, 288 + 1, 288 + 2, 7 * 288, 7 * 288 + 1, 7 * 288 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [32]:
import time
rank = 10
pred_step = 7 * 288
time_lags = np.array([1, 2, 3, 288, 289, 290, 7 * 288, 7 * 288 + 1, 7 * 288 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=1008)

Iter: 500
MAPE: 0.110953
RMSE: 8.39262

Iter: 1000
MAPE: 0.100398
RMSE: 6.22308

Imputation MAPE: 0.0943361
Imputation RMSE: 5.49515

Prediction MAPE: 0.109344
Prediction RMSE: 5.95936

Running time: 4668 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=504)

Iter: 500
MAPE: 0.0939913
RMSE: 5.4574

Iter: 1000
MAPE: 0.0925024
RMSE: 5.40891

Imputation MAPE: 0.0930147
Imputation RMSE: 5.42341

Prediction MAPE: 0.111674
Prediction RMSE: 6.03488

Running time: 2999 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.0943425
RMSE: 5.47328

Iter: 1000
MAPE: 0.0944343
RMSE: 5.49189

Imputation MAPE: 0.0940909
Imputation RMSE: 5.48087

Prediction MAPE: 0.147405
Prediction RMSE: 8.22826

Running time: 1801 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.0946713
RMSE: 5.48873

Iter: 1000
MAPE: 0.0930694
RMSE: 5.43538

Imputation MAPE: 0.0930566
Imputation RMSE: 5.43729

Prediction MAPE: 0.146669
Prediction RMSE: 8.02032

Running time: 5242 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=38)

Iter: 500
MAPE: 0.126667
RMSE: 10.0194

Iter: 1000
MAPE: 0.0948734
RMSE: 5.69245

Imputation MAPE: 0.0945119
Imputation RMSE: 5.49562

Prediction MAPE: 0.145766
Prediction RMSE: 8.13409

Running time: 1869 seconds



**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Random missing (RM)
- 40% missing rate


In [33]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values
missing_rate = 0.4

## Random missing (RM)
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 288
- Time lags: {1, 2, 288, 288 + 1, 288 + 2, 7 * 288, 7 * 288 + 1, 7 * 288 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [34]:
import time
rank = 10
pred_step = 7 * 288
time_lags = np.array([1, 2, 3, 288, 289, 290, 7 * 288, 7 * 288 + 1, 7 * 288 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=1008)

Iter: 500
MAPE: 0.0877406
RMSE: 5.10053

Iter: 1000
MAPE: 0.0858127
RMSE: 5.01607

Imputation MAPE: 0.085745
Imputation RMSE: 5.01486

Prediction MAPE: 0.10521
Prediction RMSE: 5.68526

Running time: 7172 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=504)

Iter: 500
MAPE: 0.0877946
RMSE: 5.10282

Iter: 1000
MAPE: 0.086276
RMSE: 5.03992

Imputation MAPE: 0.08575
Imputation RMSE: 5.01635

Prediction MAPE: 0.109627
Prediction RMSE: 5.89403

Running time: 4134 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.0881051
RMSE: 5.11549

Iter: 1000
MAPE: 0.0870474
RMSE: 5.08169

Imputation MAPE: 0.0868271
Imputation RMSE: 5.06591

Prediction MAPE: 0.14336
Prediction RMSE: 7.8961

Running time: 1855 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.0878311
RMSE: 5.10429

Iter: 1000
MAPE: 0.0861349
RMSE: 5.03379

Imputation MAPE: 0.0857392
Imputation RMSE: 5.01569

Prediction MAPE: 0.142843
Prediction RMSE: 7.5767

Running time: 1831 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=38)

Iter: 500
MAPE: 0.0876245
RMSE: 5.09225

Iter: 1000
MAPE: 0.0856452
RMSE: 5.00769

Imputation MAPE: 0.0857446
Imputation RMSE: 5.01483

Prediction MAPE: 0.142048
Prediction RMSE: 7.57636

Running time: 1843 seconds



**Scenario setting**:

- Tensor size: $323\times 28\times 288$ (road segment, day, time of day)
- Random missing (RM)
- 60% missing rate


In [35]:
import pandas as pd

dense_mat = pd.read_csv('../datasets/Seattle-data-set/mat.csv', index_col = 0)
RM_mat = pd.read_csv('../datasets/Seattle-data-set/RM_mat.csv', index_col = 0)
dense_mat = dense_mat.values
RM_mat = RM_mat.values
missing_rate = 0.6

## Random missing (RM)
binary_mat = np.round(RM_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 288
- Time lags: {1, 2, 288, 288 + 1, 288 + 2, 7 * 288, 7 * 288 + 1, 7 * 288 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [36]:
import time
rank = 10
pred_step = 7 * 288
time_lags = np.array([1, 2, 3, 288, 289, 290, 7 * 288, 7 * 288 + 1, 7 * 288 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 12, 18, 24, 30, 36, 42, 48, 54]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=1008)

Iter: 500
MAPE: 0.0894214
RMSE: 5.16587

Iter: 1000
MAPE: 0.0879184
RMSE: 5.12837

Imputation MAPE: 0.0879216
Imputation RMSE: 5.12756

Prediction MAPE: 0.115999
Prediction RMSE: 6.3022

Running time: 5951 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=504)

Iter: 500
MAPE: 0.0894028
RMSE: 5.16909

Iter: 1000
MAPE: 0.0878924
RMSE: 5.12459

Imputation MAPE: 0.0878341
Imputation RMSE: 5.11729

Prediction MAPE: 0.112092
Prediction RMSE: 6.09629

Running time: 3766 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.0888348
RMSE: 5.14931

Iter: 1000
MAPE: 0.0878914
RMSE: 5.12533

Imputation MAPE: 0.0878466
Imputation RMSE: 5.12046

Prediction MAPE: 0.14903
Prediction RMSE: 8.42847

Running time: 1989 seconds

Prediction time horizon (delta) = 48.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.0895034
RMSE: 5.17518

Iter: 1000
MAPE: 0.0879062
RMSE: 5.12732

Imputation MAPE: 0.0879073
Imputation RMSE: 5.12631

Prediction MAPE: 0.152438
Prediction RMSE: 8.61268

Running time: 1957 seconds

Prediction time horizon (delta) = 54.


IntProgress(value=0, max=38)

Iter: 500
MAPE: 0.0891742
RMSE: 5.16119

Iter: 1000
MAPE: 0.0879131
RMSE: 5.12778

Imputation MAPE: 0.087906
Imputation RMSE: 5.12696

Prediction MAPE: 0.15388
Prediction RMSE: 8.67315

Running time: 1977 seconds



## Evaluation on London Movement Speed Data

London movement speed data set is is a city-wide hourly traffic speeddataset collected in London.

- Collected from 200,000+ road segments.
- 720 time points in April 2019.
- 73% missing values in the original data.

|  Observation rate | $>90\%$ | $>80\%$ | $>70\%$ | $>60\%$ | $>50\%$ |
|:------------------|--------:|--------:|--------:|--------:|--------:|
|**Number of roads**|  17,666 |  27,148 |  35,912 |  44,352 |  52,727 |


If want to test on the full dataset, you could consider the following setting for masking observations as missing values. 

```python
import numpy as np
np.random.seed(1000)
mask_rate = 0.20

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
pos_obs = np.where(dense_mat != 0)
num = len(pos_obs[0])
sample_ind = np.random.choice(num, size = int(mask_rate * num), replace = False)
sparse_mat = dense_mat.copy()
sparse_mat[pos_obs[0][sample_ind], pos_obs[1][sample_ind]] = 0
```

Notably, you could also consider to evaluate the model on a subset of the data with the following setting.

In [37]:
import numpy as np
import warnings
warnings.simplefilter('ignore')

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]
sparse_mat = dense_mat.copy()

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [38]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.112064
Prediction RMSE: 2.66898

Running time: 9216 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.113649
Prediction RMSE: 2.70382

Running time: 7304 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.12204
Prediction RMSE: 2.87066

Running time: 6142 seconds

Prediction time horizon (delta) = 13.


IntProgress(value=0, max=13)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.121123
Prediction RMSE: 2.86164

Running time: 6025 seconds

Prediction time horizon (delta) = 14.


IntProgress(value=0, max=12)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.120231
Prediction RMSE: 2.82041

Running time: 6075 seconds



In [39]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.4

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Non-random missing (NM)
binary_mat = np.zeros(dense_mat.shape)
random_mat = np.random.rand(dense_mat.shape[0], 30)
for i1 in range(dense_mat.shape[0]):
    for i2 in range(30):
        binary_mat[i1, i2 * 24 : (i2 + 1) * 24] = np.round(random_mat[i1, i2] + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [40]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: 0.0955807
RMSE: 2.36991

Iter: 1000
MAPE: 0.0955983
RMSE: 2.36957

Imputation MAPE: 0.0955995
Imputation RMSE: 2.36954

Prediction MAPE: 0.113749
Prediction RMSE: 2.70763

Running time: 8733 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.0955822
RMSE: 2.37031

Iter: 1000
MAPE: 0.0955977
RMSE: 2.36953

Imputation MAPE: 0.0956001
Imputation RMSE: 2.36963

Prediction MAPE: 0.115534
Prediction RMSE: 2.73643

Running time: 6439 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.0955899
RMSE: 2.37017

Iter: 1000
MAPE: 0.0955948
RMSE: 2.36947

Imputation MAPE: 0.0955995
Imputation RMSE: 2.36951

Prediction MAPE: 0.122993
Prediction RMSE: 2.88612

Running time: 6348 seconds

Prediction time horizon (delta) = 13.


IntProgress(value=0, max=13)

Iter: 500
MAPE: 0.0955772
RMSE: 2.3702

Iter: 1000
MAPE: 0.0955985
RMSE: 2.36951

Imputation MAPE: 0.0956007
Imputation RMSE: 2.36953

Prediction MAPE: 0.12197
Prediction RMSE: 2.87591

Running time: 6146 seconds

Prediction time horizon (delta) = 14.


IntProgress(value=0, max=12)

Iter: 500
MAPE: 0.0955815
RMSE: 2.37025

Iter: 1000
MAPE: 0.0955965
RMSE: 2.36949

Imputation MAPE: 0.0955993
Imputation RMSE: 2.36962

Prediction MAPE: 0.122158
Prediction RMSE: 2.8496

Running time: 6148 seconds



In [41]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.4

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [42]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: 0.0935184
RMSE: 2.31694

Iter: 1000
MAPE: 0.0934989
RMSE: 2.31593

Imputation MAPE: 0.0935035
Imputation RMSE: 2.31592

Prediction MAPE: 0.114061
Prediction RMSE: 2.69779

Running time: 9966 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.093512
RMSE: 2.31702

Iter: 1000
MAPE: 0.0935004
RMSE: 2.31591

Imputation MAPE: 0.0935027
Imputation RMSE: 2.31594

Prediction MAPE: 0.115363
Prediction RMSE: 2.72548

Running time: 7010 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.0934987
RMSE: 2.31665

Iter: 1000
MAPE: 0.093498
RMSE: 2.31591

Imputation MAPE: 0.0935042
Imputation RMSE: 2.31591

Prediction MAPE: 0.123441
Prediction RMSE: 2.8868

Running time: 6533 seconds

Prediction time horizon (delta) = 13.


IntProgress(value=0, max=13)

Iter: 500
MAPE: 0.0935135
RMSE: 2.3168

Iter: 1000
MAPE: 0.0934987
RMSE: 2.3159

Imputation MAPE: 0.0935042
Imputation RMSE: 2.31596

Prediction MAPE: 0.124113
Prediction RMSE: 2.89357

Running time: 6569 seconds

Prediction time horizon (delta) = 14.


IntProgress(value=0, max=12)

Iter: 500
MAPE: 0.0935095
RMSE: 2.31713

Iter: 1000
MAPE: 0.0935015
RMSE: 2.31592

Imputation MAPE: 0.0934997
Imputation RMSE: 2.31588

Prediction MAPE: 0.12211
Prediction RMSE: 2.83963

Running time: 6358 seconds



In [43]:
import numpy as np
np.random.seed(1000)

missing_rate = 0.6

dense_mat = np.load('../datasets/London-data-set/hourly_speed_mat.npy')
binary_mat = dense_mat.copy()
binary_mat[binary_mat != 0] = 1
pos = np.where(np.sum(binary_mat, axis = 1) > 0.7 * binary_mat.shape[1])
dense_mat = dense_mat[pos[0], :]

## Random missing (RM)
random_mat = np.random.rand(dense_mat.shape[0], dense_mat.shape[1])
binary_mat = np.round(random_mat + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, binary_mat)

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [44]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: 0.0948773
RMSE: 2.35032

Iter: 1000
MAPE: 0.0948515
RMSE: 2.34766

Imputation MAPE: 0.094856
Imputation RMSE: 2.34768

Prediction MAPE: 0.115277
Prediction RMSE: 2.71967

Running time: 9819 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.094874
RMSE: 2.35071

Iter: 1000
MAPE: 0.0948503
RMSE: 2.34767

Imputation MAPE: 0.0948571
Imputation RMSE: 2.34771

Prediction MAPE: 0.116888
Prediction RMSE: 2.75308

Running time: 6995 seconds

Prediction time horizon (delta) = 6.


...

Iter: 500
MAPE: 0.094896
RMSE: 2.34975

Iter: 1000
MAPE: 0.0948499
RMSE: 2.34763

Imputation MAPE: 0.0948578
Imputation RMSE: 2.34781

Prediction MAPE: 0.123679
Prediction RMSE: 2.88189

Running time: 6379 seconds

Prediction time horizon (delta) = 13.


IntProgress(value=0, max=13)

Iter: 500
MAPE: 0.0948738
RMSE: 2.35031

Iter: 1000
MAPE: 0.0948537
RMSE: 2.34766

Imputation MAPE: 0.0948556
Imputation RMSE: 2.34766

Prediction MAPE: 0.123727
Prediction RMSE: 2.88436

Running time: 6377 seconds

Prediction time horizon (delta) = 14.


IntProgress(value=0, max=12)

Iter: 500
MAPE: 0.0948892
RMSE: 2.34991

Iter: 1000
MAPE: 0.0948523
RMSE: 2.34772

Imputation MAPE: 0.0948564
Imputation RMSE: 2.34763

Prediction MAPE: 0.124591
Prediction RMSE: 2.86996

Running time: 6322 seconds



## Eavluation on NYC Taxi Flow Data

**Scenario setting**:

- Tensor size: $30\times 30\times 1461$ (origin, destination, time)
- Test on original data


In [13]:
import scipy.io
import warnings
warnings.simplefilter('ignore')

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
rm_tensor = scipy.io.loadmat('../datasets/NYC-data-set/rm_tensor.mat')['rm_tensor']
sparse_tensor = dense_tensor.copy()

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [14]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.487241
Prediction RMSE: 5.14914

Running time: 521 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.486761
Prediction RMSE: 5.18459

Running time: 439 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=28)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.496123
Prediction RMSE: 5.22329

Running time: 414 seconds



**Scenario setting**:

- Tensor size: $30\times 30\times 1461$ (origin, destination, time)
- Random missing (RM)
- 40% missing rate


In [15]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
rm_tensor = scipy.io.loadmat('../datasets/NYC-data-set/rm_tensor.mat')['rm_tensor']
missing_rate = 0.4

## Random missing (RM)
binary_tensor = np.round(rm_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [16]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: 0.446469
RMSE: 4.71719

Iter: 1000
MAPE: 0.449119
RMSE: 4.70195

Imputation MAPE: 0.449271
Imputation RMSE: 4.70246

Prediction MAPE: 0.452469
Prediction RMSE: 6.05268

Running time: 550 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.4598
RMSE: 4.72689

Iter: 1000
MAPE: 0.457832
RMSE: 4.67457

Imputation MAPE: 0.458049
Imputation RMSE: 4.67604

Prediction MAPE: 0.491534
Prediction RMSE: 5.27957

Running time: 522 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=28)

Iter: 500
MAPE: 0.461772
RMSE: 4.76926

Iter: 1000
MAPE: 0.457162
RMSE: 4.68225

Imputation MAPE: 0.458122
Imputation RMSE: 4.67552

Prediction MAPE: 0.491196
Prediction RMSE: 5.29813

Running time: 461 seconds



**Scenario setting**:

- Tensor size: $30\times 30\times 1461$ (origin, destination, time)
- Random missing (RM)
- 60% missing rate


In [17]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
rm_tensor = scipy.io.loadmat('../datasets/NYC-data-set/rm_tensor.mat')['rm_tensor']
missing_rate = 0.6

## Random missing (RM)
binary_tensor = np.round(rm_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [18]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: 0.452562
RMSE: 4.86466

Iter: 1000
MAPE: 0.451275
RMSE: 4.79996

Imputation MAPE: 0.451799
Imputation RMSE: 4.77326

Prediction MAPE: 0.467131
Prediction RMSE: 6.99937

Running time: 571 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.464563
RMSE: 4.85273

Iter: 1000
MAPE: 0.466617
RMSE: 4.83975

Imputation MAPE: 0.466058
Imputation RMSE: 4.83361

Prediction MAPE: 0.504263
Prediction RMSE: 5.41257

Running time: 527 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=28)

Iter: 500
MAPE: 0.464771
RMSE: 4.86106

Iter: 1000
MAPE: 0.467557
RMSE: 4.8391

Imputation MAPE: 0.467422
Imputation RMSE: 4.83645

Prediction MAPE: 0.488555
Prediction RMSE: 5.45425

Running time: 445 seconds



**Scenario setting**:

- Tensor size: $30\times 30\times 1461$ (origin, destination, time)
- Non-random missing (NM)
- 40% missing rate


In [19]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
nm_tensor = scipy.io.loadmat('../datasets/NYC-data-set/nm_tensor.mat')['nm_tensor']
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        for i3 in range(61):
            binary_tensor[i1, i2, i3 * 24 : (i3 + 1) * 24] = np.round(nm_tensor[i1, i2, i3] + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

**Model setting**:

- Low rank: 10
- Total (rolling) prediction horizons: 7 * 24
- Time lags: {1, 2, 24, 24 + 1, 24 + 2, 7 * 24, 7 * 24 + 1, 7 * 24 + 2}
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [20]:
import time
rank = 10
pred_step = 7 * 24
time_lags = np.array([1, 2, 3, 24, 25, 26, 7 * 24, 7 * 24 + 1, 7 * 24 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=84)

Iter: 500
MAPE: 0.450246
RMSE: 4.76889

Iter: 1000
MAPE: 0.451837
RMSE: 4.72913

Imputation MAPE: 0.45194
Imputation RMSE: 4.72952

Prediction MAPE: 0.452262
Prediction RMSE: 6.079

Running time: 484 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=42)

Iter: 500
MAPE: 0.463605
RMSE: 4.77108

Iter: 1000
MAPE: 0.462584
RMSE: 4.70658

Imputation MAPE: 0.462855
Imputation RMSE: 4.70524

Prediction MAPE: 0.496072
Prediction RMSE: 5.24456

Running time: 435 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=28)

Iter: 500
MAPE: 0.46323
RMSE: 4.77853

Iter: 1000
MAPE: 0.462437
RMSE: 4.70637

Imputation MAPE: 0.462556
Imputation RMSE: 4.70679

Prediction MAPE: 0.501537
Prediction RMSE: 5.28949

Running time: 409 seconds



## Evaluation on Pacific Surface Temperature Data

**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (location x, location y, month)
- Test on original data

In [33]:
import numpy as np
import warnings
warnings.simplefilter('ignore')

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
dense_tensor[np.where(dense_tensor == 124)] = 0 # Outliers
sparse_tensor = dense_tensor.copy()

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

In [34]:
import time
rank = 10
pred_step = 10 * 12
time_lags = np.array([1, 2, 3, 12, 13, 14, 2 * 12, 2 * 12 + 1, 2 * 12 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=60)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.195215
Prediction RMSE: 27.9971

Running time: 338 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=30)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.225428
Prediction RMSE: 28.0505

Running time: 296 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=20)

Iter: 500
MAPE: nan
RMSE: nan

Iter: 1000
MAPE: nan
RMSE: nan

Imputation MAPE: nan
Imputation RMSE: nan

Prediction MAPE: 0.308151
Prediction RMSE: 31.6967

Running time: 273 seconds



**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (location x, location y, month)
- Random missing (RM)
- 40% missing rate

In [35]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
dense_tensor[np.where(dense_tensor == 124)] = 0 # Outliers
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], dense_tensor.shape[2])
missing_rate = 0.4

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

In [36]:
import time
rank = 10
pred_step = 10 * 12
time_lags = np.array([1, 2, 3, 12, 13, 14, 2 * 12, 2 * 12 + 1, 2 * 12 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=60)

Iter: 500
MAPE: 0.193902
RMSE: 28.1456

Iter: 1000
MAPE: 0.20528
RMSE: 28.4193

Imputation MAPE: 0.206016
Imputation RMSE: 28.3646

Prediction MAPE: 0.371942
Prediction RMSE: 33.9876

Running time: 330 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=30)

Iter: 500
MAPE: 0.203516
RMSE: 28.3264

Iter: 1000
MAPE: 0.211315
RMSE: 28.5503

Imputation MAPE: 0.210981
Imputation RMSE: 28.5345

Prediction MAPE: 0.208465
Prediction RMSE: 28.2134

Running time: 308 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=20)

Iter: 500
MAPE: 0.197206
RMSE: 27.8075

Iter: 1000
MAPE: 0.192091
RMSE: 28.128

Imputation MAPE: 0.192935
Imputation RMSE: 28.1008

Prediction MAPE: 0.290076
Prediction RMSE: 36.8733

Running time: 288 seconds



**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (location x, location y, month)
- Random missing (RM)
- 60% missing rate

In [41]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
dense_tensor[np.where(dense_tensor == 124)] = 0 # Outliers
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], dense_tensor.shape[2])
missing_rate = 0.6

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

In [43]:
import time
rank = 10
pred_step = 10 * 12
time_lags = np.array([1, 2, 3, 12, 13, 14, 2 * 12, 2 * 12 + 1, 2 * 12 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=60)

Iter: 500
MAPE: 0.205445
RMSE: 29.2008

Iter: 1000
MAPE: 0.193745
RMSE: 29.3143

Imputation MAPE: 0.190921
Imputation RMSE: 29.1588

Prediction MAPE: 0.250512
Prediction RMSE: 28.9742

Running time: 368 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=30)

Iter: 500
MAPE: 0.194299
RMSE: 28.6794

Iter: 1000
MAPE: 0.190711
RMSE: 28.9381

Imputation MAPE: 0.190226
Imputation RMSE: 28.8867

Prediction MAPE: 0.227131
Prediction RMSE: 29.5591

Running time: 319 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=20)

Iter: 500
MAPE: 0.191658
RMSE: 29.1911

Iter: 1000
MAPE: 0.192367
RMSE: 29.397

Imputation MAPE: 0.194123
Imputation RMSE: 29.3552

Prediction MAPE: 0.202338
Prediction RMSE: 29.0306

Running time: 299 seconds



**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (location x, location y, month)
- Non-random missing (NM)
- 40% missing rate

In [44]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
dense_tensor[np.where(dense_tensor == 124)] = 0 # Outliers
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], int(dense_tensor.shape[2] / 3))
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        for i3 in range(int(dense_tensor.shape[2] / 3)):
            binary_tensor[i1, i2, i3 * 3 : (i3 + 1) * 3] = np.round(random_tensor[i1, i2, i3] + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

dim1, dim2, dim3 = dense_tensor.shape
dense_mat = np.zeros((dim1 * dim2, dim3))
sparse_mat = np.zeros((dim1 * dim2, dim3))
for i in range(dim1):
    dense_mat[i * dim2 : (i + 1) * dim2, :] = dense_tensor[i, :, :]
    sparse_mat[i * dim2 : (i + 1) * dim2, :] = sparse_tensor[i, :, :]

In [46]:
import time
rank = 10
pred_step = 10 * 12
time_lags = np.array([1, 2, 3, 12, 13, 14, 2 * 12, 2 * 12 + 1, 2 * 12 + 2])
burn_iter = 1000
gibbs_iter = 200
for multi_step in [2, 4, 6]:
    start = time.time()
    print('Prediction time horizon (delta) = {}.'.format(multi_step))
    mat_hat = BTMF_forecast(dense_mat, sparse_mat, pred_step, multi_step, rank, time_lags, burn_iter, gibbs_iter)
    end = time.time()
    print('Running time: %d seconds'%(end - start))
    print()

Prediction time horizon (delta) = 2.


IntProgress(value=0, max=60)

Iter: 500
MAPE: 0.1876
RMSE: 27.5965

Iter: 1000
MAPE: 0.193474
RMSE: 27.7275

Imputation MAPE: 0.194085
Imputation RMSE: 27.6665

Prediction MAPE: 0.2091
Prediction RMSE: 28.2023

Running time: 383 seconds

Prediction time horizon (delta) = 4.


IntProgress(value=0, max=30)

Iter: 500
MAPE: 0.195747
RMSE: 27.6092

Iter: 1000
MAPE: 0.192273
RMSE: 27.8356

Imputation MAPE: 0.193498
Imputation RMSE: 27.7933

Prediction MAPE: 0.192317
Prediction RMSE: 27.8698

Running time: 319 seconds

Prediction time horizon (delta) = 6.


IntProgress(value=0, max=20)

Iter: 500
MAPE: 0.198317
RMSE: 28.0973

Iter: 1000
MAPE: 0.211678
RMSE: 28.2329

Imputation MAPE: 0.212003
Imputation RMSE: 28.2287

Prediction MAPE: 0.20921
Prediction RMSE: 28.0786

Running time: 314 seconds



### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>